In [27]:
import os, sys, inspect, io

cmd_folder = os.path.realpath(
    os.path.dirname(
        os.path.abspath(os.path.split(inspect.getfile( inspect.currentframe() ))[0])))

if cmd_folder not in sys.path:
    sys.path.insert(0, cmd_folder)
    
from transitions import *
from transitions.extensions import GraphMachine
from IPython.display import Image, display, display_png
from queue import Queue

class Optimizer(object):
    q_in = Queue()
    q_out = Queue()
    
    def do_work(self, item):
        return item + 5
    
    def on_enter_training(self):
        item = self.q_in.get()
        self.q_out.put(self.do_work(item))
        self.q_in.task_done()
        self.trained()
        
    def on_exit_training(self):
        item = self.q_out.get()
        print("result of training {}".format(item))
        
    def on_enter_listening(self):
        self.q_in.put(1)
        self.heard()
        
    def is_valid(self):
        return True
    
    def is_not_valid(self):
        return False
    
    def is_also_valid(self):
        return True
    
    # graph object is created by the machine
    def show_graph(self, **kwargs):
        stream = io.BytesIO()
        self.get_graph(**kwargs).draw(stream, prog='dot', format='png')
        display(Image(stream.getvalue()))

In [28]:
transitions = [
    { 'trigger': 'work_done', 'source': ['training', 'averaging'], 'dest': 'broadcasting' },
    { 'trigger': 'sent', 'source': 'broadcasting', 'dest': 'listening', 'conditions':'is_valid' },
    { 'trigger': 'heard', 'source': 'listening', 'dest': 'training', 'unless':'is_not_valid' }
#     { 'trigger': 'averaged', 'source': 'averaging', 'dest': 'broadcasting',
#       'conditions':['is_valid','is_also_valid'] }
]
states=['training', 'broadcasting', 'listening', 'averaging']

model = Optimizer()
machine = Machine(model=model, 
                       states=states, 
                       transitions=transitions,
                       initial='broadcasting')
model.state

'broadcasting'

In [5]:
class FederatedAveragingOptimizer(Machine):
    q_in = Queue()
    q_out = Queue()
    
    def do_work(self, item):
        return item + 5
    
    def on_enter_training(self):
        item = self.q_in.get()
        self.q_out.put(self.do_work(item))
        self.q_in.task_done()
        self.work_done()
        
    def on_exit_training(self):
        item = self.q_out.get()
        print("result of training {}".format(item))
        
    def on_enter_listening(self):
        self.q_in.put(1)
        self.heard()
        
    def is_valid(self):
        return True
    
    def is_not_valid(self):
        return False
    
    def is_also_valid(self):
        return True
    
    def __init__(self):
        transitions = [
        { 'trigger': 'work_done', 'source': ['training', 'averaging'], 'dest': 'broadcasting' },
        { 'trigger': 'sent', 'source': 'broadcasting', 'dest': 'listening', 'conditions':'is_valid' },
        { 'trigger': 'heard', 'source': 'listening', 'dest': 'training', 'unless':'is_not_valid' }
        #     { 'trigger': 'averaged', 'source': 'averaging', 'dest': 'broadcasting',
        #       'conditions':['is_valid','is_also_valid'] }
        ]
        states=['training', 'broadcasting', 'listening', 'averaging']
        Machine.__init__(self, states=states, transitions=transitions, initial='broadcasting')
        

In [6]:
fedavg = FederatedAveragingOptimizer()
fedavg.state
fedavg.sent()

result of training 6


True

In [25]:
class P2POptimizer(Machine):
    
    def do_averaging(self, event):
        self.neural_network += 'trained'
        print(self.neural_network)
        
    def get_model_with_addr(self, model_addr):
        return "hello im a model"
    
    def on_enter_training(self, event):
        model_addr = event.kwargs.get('model_addr')
        self.neural_network = self.get_model_with_addr(model_addr)
        print(self.model)
        
    def on_exit_training(self, event):
        print("result of training {}".format(self.neural_network))
        
    def is_valid(self, event):
        return True
    
    def time_elapsed(self, event):
        return self.time >= self.time_threshold
    
    def is_also_valid(self):
        return True
    
    
    def __init__(self):
        self.time = 0
        self.time_threshold = 2
        self.neural_network = False
        transitions = [
        { 'trigger': 'trained', 'source': 'training', 'dest': 'broadcasting'},
        { 'trigger': 'averaged', 'source': 'averaging', 'dest': 'training' },
        { 'trigger': 'sent', 'source': 'broadcasting', 'dest': 'listening', 'conditions':'is_valid' },
        { 'trigger': 'heard', 'source': 'listening', 'dest': 'averaging', 'conditions':'time_elapsed', 'after':'do_averaging' }
        #     { 'trigger': 'averaged', 'source': 'averaging', 'dest': 'broadcasting',
        #       'conditions':['is_valid','is_also_valid'] }
        ]
        states=['listening', 'averaging', 'training', 'broadcasting']
        Machine.__init__(self, states=states, transitions=transitions, send_event=True, initial='broadcasting')

In [26]:
fclyr = P2POptimizer()
fclyr.sent()
fclyr.heard(model_addr='0xabc123')

False